In [1]:
from tfl.client import Client
from tfl.api_token import ApiToken

app_id = 'Subscription_Peterson'
app_key = '8d1f8ab05c7c49db81dd99c40f70574c'

token = ApiToken(app_id, app_key)

client = Client(token)
#print (client.get_line_meta_modes())
#print (client.get_lines(mode="bus")[0])
#print (client.get_lines(line_id="victoria")[0])



In [2]:
import requests
from collections import defaultdict
import time


In [3]:
import urllib.request, json
import requests

try:
    url = "https://api.tfl.gov.uk/crowding/940GZZLUSKS/Live?app_key=8d1f8ab05c7c49db81dd99c40f70574c"
    hdr ={
    # Request headers
    'Cache-Control': 'no-cache',
    }

    req = urllib.request.Request(url, headers=hdr)
    req.get_method = lambda: 'GET'
    response = urllib.request.urlopen(req)
    print(response.getcode())
#    print(response.read())
except Exception as e:
    print(e)
    
    

200


In [4]:
print(response)



In [5]:
import pandas as pd


In [6]:
import sys

class Line:    
       
    def __init__(self,name,color):
        self.name = name
        self.color = color
        
        
class Station(Line):
    
    def __init__(self,linename,color,naptan,commonname,latitude,longitude):
        super().__init__(linename,color)
        self.naptan = naptan

    
bakerloo = Line("bakerloo","#B36305")
central = Line("central","#E32017")
circle =  Line("circle","#FFD300")
district = Line("district","#00782A")
hammersmith = Line("hammersmith-city","#F3A9BB")
jubilee = Line("jubilee","#A0A5A9")
metropolitan = Line("metropolitan","#9B0056")
northern = Line("northern","#000000")
piccadilly = Line("piccadilly","#003688")
victoria = Line("victoria","#0098D4")
waterloo = Line("waterloo-city","#95CDBA")

tube_lines = [bakerloo,central,circle,district,hammersmith,jubilee,metropolitan,northern,piccadilly,victoria,waterloo]
#tube_lines.append(bakerloo = Line("bakerloo","#B36305")


#tube_lines = [("bakerloo","#B36305"),("central","#E32017"),("circle","#FFD300"),("district","#00782A"),("hammersmith-city","#F3A9BB"),("jubilee","#A0A5A9"),("metropolitan","#9B0056"),("northern","#000000"),("piccadilly","#003688"),("victoria","#0098D4"),("waterloo-city","#95CDBA")]


In [7]:
# use requests to interact with an API
import zipfile, io

# The API path should be stated in the documentation 

api_path = "https://api.tfl.gov.uk/crowding/940GZZLUSKS/Live?app_key=8d1f8ab05c7c49db81dd99c40f70574c"

stations = defaultdict(list)

for line in tube_lines:
    api_path1 = "https://api.tfl.gov.uk/Line/%s/StopPoints" % line.name
    r1 = requests.get(api_path1).json()
    
    for entry in r1:
        stations[entry["naptanId"]] = Station(line.name, line.color, entry["naptanId"], entry["commonName"][:-20],0,0)     
        
url_coord = "https://api.tfl.gov.uk/stationdata/tfl-stationdata-detailed.zip"

r = requests.get(url_coord)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("./details")

df = pd.read_csv("./details/StationPoints.csv")

req = urllib.request.Request(url_coord, headers=hdr)

req.get_method = lambda: 'GET'
response = urllib.request.urlopen(req)


In [8]:
i = 0
for stop in df.StationUniqueId:
    i +=1
    if stop in list(stations.keys()) and df.FriendlyName.iloc[i].startswith(('Sub','Met','MET')):
        
        stations[stop].latitude = df.Lat.iloc[i]
        stations[stop].longitude = df.Lon.iloc[i]             

In [9]:
stations['940GZZLUBST'].latitude

51.52285

In [10]:
from datetime import datetime, timedelta

time_list = []
start_time = datetime.strptime("00:00","%H:%M")

time_interval = timedelta(minutes=15)

while start_time.strftime("%H:%M") != "00:00" or len(time_list) == 0:
    time_list.append(start_time.strftime("%H:%M"))
    start_time += time_interval
    

In [11]:
day_station = {'Mon':{},'Tue':{},'Wed':{},'Thu':{},"Fri":{},'Sat':{},'Sun':{}}

for day in day_station.keys():
    
    for interval in range(95):
        day_station[day][time_list[interval]] = {}
    
    for station in stations.keys():    
        api_path = "https://api.tfl.gov.uk/crowding/%s/%s?app_key=8d1f8ab05c7c49db81dd99c40f70574c" % (station, day)
    
        r = requests.get(api_path).json()

        for interval in range(95):
    
#        for station in stations.keys():         
            try:
                #print(r["timeBands"][1]["percentageOfBaseLine"])
                day_station[day][time_list[interval]][station] = r["timeBands"][interval]["percentageOfBaseLine"]
            except:
                continue
        
        #print(r['percentageOfBaseline'])
        

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
from geopy.geocoders import ArcGIS
import folium

import ipywidgets as widgets
from IPython.display import display, clear_output

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="location_details")

location = geolocator.geocode("London")
print((location.latitude, location.longitude))

selected_day = 'Mon'
selected_time = 0

def create_map(dayofweek,interval):
    # Clear previous output to avoid multiple maps showing up
    clear_output(wait=True)
    
    # Create a map centered on the initial position
    mymap = folium.Map(location=[location.latitude, location.longitude], zoom_start=12)

    # Add markers based on the current time_step
    for naptan, station in stations.items():
        try:
            coef = day_station[dayofweek][interval][naptan]
            popup_info = station.commonname+'\n'+str(coef)
            try:
                folium.Circle(location=[station.latitude,station.longitude],radius=500*coef,popup=popup_info,color=station.color,fill=True,fill_color=station.color,fill_opacity=0.4).add_to(mymap)
            except:
                continue
        except:
            continue
    display(day_buttons)
    display(min15_slider)
    display(mymap)
    

In [ ]:
min15_slider = widgets.SelectionSlider(
    options=time_list,
    value=time_list[0],  # Default value
    description='Time:',
    continuous_update=False  # Update map only when slider is released
)

day_buttons = widgets.ToggleButtons(
    options=list(day_station.keys()),
    description='Day:',
    button_style='',  # Default style
)


day_buttons.style.button_width = '70px'  # Width of each button
day_buttons.style.font_weight = 'bold'

def on_day_change(change):
    # Highlight the selected button
    global selected_day
    selected_day = change["new"]
    day_buttons.button_style = ''
    day_buttons.style.button_color = ''  # Reset previous styles
    create_map(selected_day, selected_time)
    
def on_time_change(change):
    global selected_time
    selected_time = change["new"]
    create_map(selected_day, selected_time)
    
day_buttons.observe(on_day_change, names='value')
min15_slider.observe(on_time_change, names='value')

display(day_buttons)
display(min15_slider)

create_map(selected_day, selected_time)

In [ ]:
mymap1=folium.Map(location=[location.latitude, location.longitude],zoom_start=12)
title_html = '''
     <h3 align="center" style="font-size:16px"><b>Live status</b></h3>
     '''
mymap1.get_root().html.add_child(folium.Element(title_html))

import folium.plugins as plugins

for naptan, station in stations.items():
    
    api_path = "https://api.tfl.gov.uk/crowding/%s/Live?app_key=8d1f8ab05c7c49db81dd99c40f70574c" % naptan
    
    r = requests.get(api_path).json()
    coef = r['percentageOfBaseline']
    popup_info = station[2]+'\n'+str(coef)
    
    try:        
        folium.Circle(location=[station[3],station[4]],radius=500*coef,popup=popup_info,color=station[1],fill=True,fill_color=station[1],fill_opacity=0.4).add_to(mymap1)
    except:
        continue

mymap1

In [ ]:
live_data2 = {}

for naptan, station in stations.items():
    live_data2[naptan] = {}
    

In [ ]:
#url = "https://api.tfl.gov.uk/crowding/940GZZLUSKS/Live?app_key=8d1f8ab05c7c49db81dd99c40f70574c"

# Headers or parameters if needed
#headers = {'Authorization': 'Bearer YOUR_ACCESS_TOKEN'}

# Define the interval (10 minutes in seconds)
interval = 5 * 60  # 10 minutes

#live_data = {}

# Function to make the REST request
def make_request():
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            # Process the data as needed
            print(f"Data received: {data}")
        else:
            print(f"Failed to retrieve data, status code: {response.status_code}")
    except requests.RequestException as e:
        print(f"Request failed: {e}")

def make_live_request():       
    for naptan, station in stations.items():
        #live_data[naptan] = {}
        api_path = "https://api.tfl.gov.uk/crowding/%s/Live?app_key=8d1f8ab05c7c49db81dd99c40f70574c" % naptan  
        try:
            response = requests.get(api_path)
            r = requests.get(api_path).json()
            if response.status_code == 200:
                time = r['timeLocal']
                coef = r['percentageOfBaseline']
                print(r)
                live_data2[naptan][time] = coef
            else:
                print(f"Failed to retrieve data, status code: {response.status_code}")
        except requests.RequestException as e:
            print(f"Request failed: {e}")
  #  print(live_data['940GZZLUSKS'])
# Loop to request data every 10 minutes
while True:
    make_live_request()
    time.sleep(interval)  # Wait for the specified interval before making the next request